In [365]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd
import numpy as np
from io import BytesIO
from datetime import datetime, timedelta
import boto3
from src.config import *
from src.s3_utils import get_s3_object

pd.set_option('display.max_columns', None)

In [366]:
file_path = r"C:\Users\jklas\Downloads\gb_day_cycle_forecast.csv"
file = pd.read_csv(file_path)

cvp_agl_path = r"C:\Users\jklas\Downloads\cvp_agl.csv"
cvp_agl_uk = pd.read_csv(cvp_agl_path)

cvp_amzl_uk= get_s3_object(scheduling_bucket, "UK_CVP_Plan_All")
file.query('cycle=="sd"').head()

au_fcst_path = r"C:\Users\jklas\Downloads\au_fcst.csv"
au_fcst = pd.read_csv(au_fcst_path)

In [ ]:


# which forecast maps to which ofd_volume (e.g. is LPV only AGL and Cat only AMZL)?


# missing SSD fcsts --> where to find?
# what is "over-night" cycle in AU?
# where to get SG forecasts?


In [368]:
unique_stations = file.query('business_type=="GSF"')["cycle"].unique()
# unique_stations = file["business_type"].unique()
# unique_stations = file["business_type"].unique()
print(unique_stations)

['day']


Maestro File Processing

In [369]:
file = file.copy()

file = file.rename(columns={
    "forecast_date": "date"
}) 

relevant_cols = [
    "station",
    "business_type",
    "cycle",
    "date",
    "forecast_car_vol",
    "forecast_lpv_vol"
]

cycle_map = {
    "cy1": "CYCLE_1",
    "cy2": "CYCLE_2",
    "sd_a": "CYCLE_SD_A",
    "sd_b": "CYCLE_SD_B",
    "sd_c": "CYCLE_SD_C",
    "adhoc": "AD_HOC",
    "day": "day",
    "brunch": "brunch",
    "lunch": "lunch",
    "bedtime": "bedtime",
    "dinner": "dinner",
    "sd": "sd"
}

file["cycle"] = file["cycle"].map(cycle_map, na_action=None)
file = file[relevant_cols]
file["date"] = pd.to_datetime(file["date"], dayfirst=False)
file["forecast_sum"] = file["forecast_car_vol"] + file["forecast_lpv_vol"]
file["car_share"] = np.where(file["forecast_sum"] == 0, 0, file["forecast_car_vol"] / file["forecast_sum"])
file["lpv_share"] = np.where(file["forecast_sum"] == 0, 0, file["forecast_lpv_vol"] / file["forecast_sum"])

file.head()

,station,business_type,cycle,date,forecast_car_vol,forecast_lpv_vol,forecast_sum,car_share,lpv_share
0,CC06,GSF,day,2025-04-27,15.576847,15.423153,31.0,0.502479,0.497521
1,CC06,GSF,day,2025-04-28,15.576847,15.423153,31.0,0.502479,0.497521
2,CC06,GSF,day,2025-04-29,14.069410,13.930590,28.0,0.502479,0.497521
3,CC06,GSF,day,2025-04-30,17.084284,16.915716,34.0,0.502479,0.497521
4,CC06,GSF,day,2025-05-01,16.079326,15.920674,32.0,0.502479,0.497521


CVP AMZL Data Processing

In [370]:
cvp_amzl_uk = cvp_amzl_uk.copy()

cvp_amzl_uk = cvp_amzl_uk.rename(columns={
    "ofd_date": "date",
    "ofd_volume": "snop_forecast"
}) 
cvp_amzl_uk["business_type"] = "AMZL"

relevant_cols = [
    "station",
    "business_type",    
    "cycle",            
    "date",    
    "snop_forecast"           
]

cvp_amzl_uk = cvp_amzl_uk[relevant_cols]
cvp_amzl_uk["date"] = pd.to_datetime(cvp_amzl_uk["date"], dayfirst=False)

cvp_amzl_uk.head()

,station,business_type,cycle,date,snop_forecast
0,DXG2,AMZL,CYCLE_1,2025-07-25,63419.023494
1,DCF1,AMZL,CYCLE_1,2025-06-17,53693.089555
2,DWR1,AMZL,CYCLE_SD_B,2025-08-13,860.850198
3,DXE1,AMZL,CYCLE_SD_C,2025-06-17,1434.000000
4,DHU2,AMZL,CYCLE_SD_A,2025-07-29,3416.800985


CVP AGL Data Processing

In [371]:
cvp_agl_uk = cvp_agl_uk.copy()

cvp_agl_uk = cvp_agl_uk.rename(columns={
    "ofd_date": "date",
    "forecast": "snop_forecast"
}) 

relevant_cols = [
    "station",
    "business_type",       
    "cycle",            
    "date",    
    "snop_forecast"           
]

cvp_agl_uk = cvp_agl_uk[relevant_cols]
cvp_agl_uk["date"] = pd.to_datetime(cvp_agl_uk["date"], dayfirst=False)
cvp_agl_uk["business_type"] = "GSF"
cvp_agl_uk["cycle"] = "day"

cvp_agl_uk.head()

,station,business_type,cycle,date,snop_forecast
0,UUK2,GSF,day,2020-06-15,496.0
1,ULI1,GSF,day,2020-07-17,303.0
2,ULI2,GSF,day,2020-10-31,496.0
3,CU09,GSF,day,2022-05-27,385.0
4,ULO5,GSF,day,2022-09-24,530.0


In [372]:
cvp_agl_uk["cycle"].unique()

array(['day'], dtype=object)

CVP AU Data

In [373]:
au_fcst = au_fcst.copy()

au_fcst = au_fcst.rename(columns={
    "forecast_delivery_date": "date",
    "pickup_location": "station",
    "dispatch_cycle": "cycle",
    "forecast_volume": "snop_forecast",
}) 

cycle_map = {
    "Cycle1": "CYCLE_1",
    "Cycle2": "CYCLE_2",
    "Same_day_1": "CYCLE_SD_A",
    "Same_day_2": "CYCLE_SD_B",
    "Over_Night": "Over_Night"
}

# au_fcst["cycle"] = au_fcst["cycle"].map(cycle_map, na_action=None)
au_fcst["date"] = pd.to_datetime(au_fcst["date"], dayfirst=False)
au_fcst = au_fcst[au_fcst["last_mile_carrier"]=="Flex"]

au_fcst[au_fcst['cycle']=="Over_Night"].count()
# au_fcst["cycle"].unique()

file_creation_date       4344
date                     4344
station                  4344
cycle                    4344
source_method_volume     4344
snop_forecast            4344
local_file_identifier    4344
last_update_date         4344
last_mile_carrier        4344
dtype: int64

Join all tables

In [374]:
# Stack the CVP + AGL forecasts

forecasts = pd.concat([
    cvp_amzl_uk,
    cvp_agl_uk
], ignore_index=True)

forecasts.head()



,station,business_type,cycle,date,snop_forecast
0,DXG2,AMZL,CYCLE_1,2025-07-25,63419.023494
1,DCF1,AMZL,CYCLE_1,2025-06-17,53693.089555
2,DWR1,AMZL,CYCLE_SD_B,2025-08-13,860.850198
3,DXE1,AMZL,CYCLE_SD_C,2025-06-17,1434.000000
4,DHU2,AMZL,CYCLE_SD_A,2025-07-29,3416.800985


In [375]:
# Date Comparison

cvp_rng = forecasts["date"].sort_values(ascending=False).unique()
print(f"""
      CVP______
      Max: {cvp_rng[0]}
      Min: {cvp_rng[-1]}
      """)

maestro_rng = file["date"].sort_values(ascending=False).unique()
print(f"""
      Maestro______
      Max: {maestro_rng[0]}
      Min: {maestro_rng[-1]}
      """)


      CVP______
      Max: 2025-09-06 00:00:00
      Min: 2020-04-01 00:00:00
      

      Maestro______
      Max: 2025-10-25 00:00:00
      Min: 2025-04-27 00:00:00
      


In [376]:
# Join tables
join_cols = ["date", "station", "cycle", "business_type"]

df = file.merge(
    forecasts,
    on= join_cols,
    how="left",
    suffixes=["_maestro", "_snop"]
)

df["difference"] = df["forecast_sum"] - df["snop_forecast"]
df["match"] = df["difference"] != 0

filter = (df["date"] > datetime.now()) & (df["date"] < (datetime.now() + timedelta(weeks=2)))

df = df[filter]


df.query('forecast_sum != 0 & snop_forecast.isna() == True') #["business_type"].unique()

,station,business_type,cycle,date,forecast_car_vol,forecast_lpv_vol,forecast_sum,car_share,lpv_share,snop_forecast,difference,match
6565,VCY2,SSD,bedtime,2025-05-31,690.424867,41.575133,732.0,0.943203,0.056797,NaN,NaN,True
6566,VCY2,SSD,bedtime,2025-06-01,715.891358,43.108642,759.0,0.943203,0.056797,NaN,NaN,True
6567,VCY2,SSD,bedtime,2025-06-02,699.856901,42.143099,742.0,0.943203,0.056797,NaN,NaN,True
6568,VCY2,SSD,bedtime,2025-06-03,706.459325,42.540675,749.0,0.943203,0.056797,NaN,NaN,True
6569,VCY2,SSD,bedtime,2025-06-04,704.572918,42.427082,747.0,0.943203,0.056797,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...
79570,VSA6,SSD,lunch,2025-06-09,2762.036590,127.963410,2890.0,0.955722,0.044278,NaN,NaN,True
79571,VSA6,SSD,lunch,2025-06-10,2569.936468,119.063532,2689.0,0.955722,0.044278,NaN,NaN,True
79572,VSA6,SSD,lunch,2025-06-11,2590.006630,119.993370,2710.0,0.955722,0.044278,NaN,NaN,True
79573,VSA6,SSD,lunch,2025-06-12,2621.545456,121.454544,2743.0,0.955722,0.044278,NaN,NaN,True


In [377]:
df.head()

,station,business_type,cycle,date,forecast_car_vol,forecast_lpv_vol,forecast_sum,car_share,lpv_share,snop_forecast,difference,match
68,CC06,GSF,day,2025-05-31,18.591721,18.408279,37.0,0.502479,0.497521,32.0,5.0,True
69,CC06,GSF,day,2025-05-31,18.591721,18.408279,37.0,0.502479,0.497521,36.0,1.0,True
70,CC06,GSF,day,2025-06-01,17.586763,17.413237,35.0,0.502479,0.497521,35.0,0.0,False
71,CC06,GSF,day,2025-06-02,16.581805,16.418195,33.0,0.502479,0.497521,33.0,0.0,False
72,CC06,GSF,day,2025-06-03,15.576847,15.423153,31.0,0.502479,0.497521,30.0,1.0,True


In [378]:
df.isna().sum()

station                0
business_type          0
cycle                  0
date                   0
forecast_car_vol       0
forecast_lpv_vol       0
forecast_sum           0
car_share              0
lpv_share              0
snop_forecast       1676
difference          1676
match                  0
dtype: int64